## Experiment 2
- v2 : 컬럼단위로 retrieve하기 위한 버전. 다음과 같이 2가지 collection을 생성
   - table : table_name, table_comment
   - column : table_name, column_name, desc_text

In [ ]:
from langchain_milvus import Milvus
from langchain_ollama import OllamaEmbeddings

from milvus.client.create_collection import create_collection
from milvus.preprocess.preprocess import load_td, preprocess_td
from milvus.preprocess.td2docs_column import td2docs_column_level, td2docs_table_comment

In [ ]:
json_path = "./data/_generated_schemas_shilla_t2s_devloper_gemma3_12b_20250604.json"
URI = "http://10.128.0.20:19530"


In [ ]:
# Convert table description to docs
data = load_td(json_path)
td = preprocess_td(data)

docs_table = td2docs_table_comment(td)
docs_column = td2docs_column_level(td)

### 1. Table

In [ ]:
embedding = OllamaEmbeddings(model="bge-m3", base_url="http://10.128.0.20:11434")
create_collection(
    docs=docs_table,
    collection_name="shilla_t2s_developer_gemma3_12b_20250604_v2_table",
    embedding=embedding,
    uri=URI,
    overwrite=True
)

### 2. Column

In [ ]:
embedding = OllamaEmbeddings(model="bge-m3", base_url="http://10.128.0.20:11434")
create_collection(
    docs=docs_column,
    collection_name="shilla_t2s_developer_gemma3_12b_20250604_v2_column",
    embedding=embedding,
    uri=URI,
    overwrite=True
)

In [ ]:
# Drop!

# from milvus.client.drop_collection import drop_collection
# drop_collection(
#     "shilla_t2s_developer_gemma3_12b_20250528_v2_column",
#     uri=URI
# )

### 3. Test

In [ ]:
# Table

vector_store_loaded = Milvus(
    embedding_function=embedding,
    collection_name="shilla_t2s_developer_gemma3_12b_20250604_v2_table",
    connection_args={"uri": URI}
)
retriever = vector_store_loaded.as_retriever(search_type="mmr", search_kwargs={"k": 3})
retriever.invoke("What is the average work time for the 정삭 process at the 장산공장 plant, produced in May 2024?")


In [ ]:
# Column

vector_store_loaded = Milvus(
    embedding_function=embedding,
    collection_name="shilla_t2s_developer_gemma3_12b_20250604_v2_column",
    connection_args={"uri": URI}
)
retriever = vector_store_loaded.as_retriever(search_type="mmr", search_kwargs={"k": 10})
retriever.invoke("What is the average work time for the 정삭 process at the 장산공장 plant, produced in May 2024?")
